# Experiment notebook

The goal of this notebook is to help me understand the code

In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[3]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import os
import torch
from pathlib import Path
import torch.nn as nn
from torchvision import datasets, transforms, models

from pytorchcv.model_provider import get_model as ptcv_get_model

from fairness_cv_project.methods.label_free_cbm.src.utils import utils, data_utils
from fairness_cv_project.methods.label_free_cbm.src.models import cbm

## Sort by alphabetical order

In [3]:
action_list = ["phoning", "hugging", "eating", "admiring", "leaning", "putting", "carrying", "reading",

In [ ]:
action_list.sort()

## Dataframe check

In [9]:
my_dict = {'male_1': {'accuracy': 0.36082474226804123, 'correct': 105, 'count': 291}, 'male_2': {'accuracy': 0.356401384083045, 'correct': 103, 'count': 289}, 'female_1': {'accuracy': 0.38831615120274915, 'correct': 113, 'count': 291}, 'female_2': {'accuracy': 0.39792387543252594, 'correct': 115, 'count': 289}}

In [10]:
import pandas as pd

pd.DataFrame(my_dict).transpose()

,accuracy,correct,count
male_1,0.360825,105.0,291.0
male_2,0.356401,103.0,289.0
female_1,0.388316,113.0,291.0
female_2,0.397924,115.0,289.0


## Check resnet model fine tuned 

In [3]:
resnet_model = models.resnet50()

In [6]:
resnet_model.fc.in_features

2048

## Check layer in Resnet

In [3]:
resnet_model = models.resnet50(pretrained=True)

/state/partition1/llgrid/pkg/anaconda/anaconda3-2023a/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2023a/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Check number of concepts of ImageNet model

In [5]:
path_imagenet_lf_cbm = Path('saved_models/imagenet_lf_cbm')
device = 'cpu'

imagenet_lf_cbm = cbm.load_cbm(path_imagenet_lf_cbm, device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/gridsan/vyuan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
imagenet_lf_cbm

CBM_model(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(6

## Look at the model

In [3]:
target_model = models.alexnet()
target_model.classifier[6] = nn.Linear(4096, 2)
state_dict = torch.load(root / f'saved_models/doctor_nurse_alexnet/alexnet_doctor_nurse.pt', map_location='cpu')

target_model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
target_model_alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
target_model_alexnet = models.alexnet()
state_dict = torch.load(root / f'saved_models/alexnet.pt', map_location='cpu')

target_model_alexnet.load_state_dict(state_dict)


<All keys matched successfully>

In [6]:
path = Path("/home/gridsan/vyuan/Label-free-CBM") / 'saved_models' / 'resnet18_cub.pt'
target_model = torch.load(path).to('cuda')

In [7]:
target_model

ResNet(
  (features): Sequential(
    (init_block): ResInitBlock(
      (conv): ConvBlock(
        (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activ): ReLU(inplace=True)
      )
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBlock(
          (conv1): ConvBlock(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): ReLU(inplace=True)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

## Understand the hook thing

In [10]:
hooks = {}
target_layers = ['layer4']
all_features = {target_layer:[] for target_layer in target_layers}
pool_mode=
hooks = {}
    for target_layer in target_layers:
        command = "target_model.{}.register_forward_hook(get_activation(all_features[target_layer], pool_mode))".format(target_layer)
        hooks[target_layer] = eval(command)

AttributeError: 'ResNet' object has no attribute 'layer4'

In [4]:
clip_name = "ViT-B/16"
target_name = "clip_RN50"
d_probe = "train"
concept_set = "data/concept_sets/cifar10_filtered.txt"
pool_mode = "avg"
save_dir = "saved_activations"


target_save_name, clip_save_name, text_save_name = get_save_names(clip_name, target_name, 
                                                                    "{}", d_probe, concept_set, 
                                                                      pool_mode, save_dir)

SyntaxError: EOL while scanning string literal (3895075241.py, line 5)

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

percent = ''

data_dir = '/home/gridsan/vyuan/Label-free-CBM/data/datasets/doctor_nurse/train_val_split'
image_datasets = {'train': datasets.ImageFolder(data_dir + str(percent) + '/train',
                                          data_transforms['train']),
                  'val': datasets.ImageFolder(data_dir + '/val',
                                          data_transforms['val'])
                  }
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [7]:
path_root = Path("/home/gridsan/vyuan/Label-free-CBM")
model_alexnet=data_utils.get_target_model('alexnet_doctor_nurse', 'cuda')

AttributeError: '_IncompatibleKeys' object has no attribute 'to'

In [9]:
type(torch.load(path_root / f'saved_models/alexnet_doctor_nurse.pt'))

collections.OrderedDict

In [14]:
target_model = models.alexnet()
target_model.classifier[6] = nn.Linear(4096, 2)
state_dict = torch.load(path_root / f'saved_models/doctor_nurse_alexnet/alexnet_doctor_nurse.pt', map_location='cpu')
# target_model = target_model.load_state_dict(state_dict)

In [12]:
type(target_model)

torch.nn.modules.module._IncompatibleKeys

In [15]:
target_model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
a =['a hospital', 'a medical chart', 'a medical degree', 'a medication', 'a name tag', "a nurse's station", 'a phone', 'a prescription pad', 'a thermometer', 'an examination room', 'expert', 'health care worker', 'human', 'professional', 'wearing scrubs']

In [4]:
for i in a:
    print(i)

a hospital
a medical chart
a medical degree
a medication
a name tag
a nurse's station
a phone
a prescription pad
a thermometer
an examination room
expert
health care worker
human
professional
wearing scrubs
